1. 버스 정류장, 카페, 고도, 인구밀도 → 각각 G2, G3, G4, G5로 가중치 구하기
2. 각 그래프의 가중치들에 대해 minmax 적용해서 정규화 및 가중치 부여 후 합산
3. $G_{total} =\sum{a_i * w_i }$
4. 쓰레기통 데이터를 G_total에 넣어서 점수 구하기(겹치는거 빼주기)
5. 우리가 생각하는 최적의 쓰레기통 배치 구해서 비교

그래프로 표현한 지도에 격자로 생각한 뒤, 특정 격자로부터 가까운 노드와 간선을 찾는 코드

In [ ]:

import networkx as nx
import osmnx as ox
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.font_manager as fm
import folium
import geopandas as gpd

G = ox.graph_from_place('서교동, 마포구 ,서울, 대한민국', network_type='walk')
fig, ax = ox.plot_graph(G)
G_proj = ox.project_graph(G)
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=False)
G2 = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)
print("Intersections: {}, G: {}, G2: {}".format(len(intersections), len(G), len(G2)))

fig, ax = ox.plot_graph(G, node_color='r')
fig, ax = ox.plot_graph(G2, node_color='r')

nodes, edges = ox.graph_to_gdfs(G)
bbox = nodes.total_bounds  # 경계를 구합니다: (minx, miny, maxx, maxy)

cell_size = 0.001  # 격자 크기 (단위: 도)
x_min, y_min, x_max, y_max = bbox
x_points = np.arange(x_min, x_max, cell_size)
y_points = np.arange(y_min, y_max, cell_size)

grid_points = [(x, y) for x in x_points for y in y_points]

example_point = grid_points[200]  # 예시로 첫 번째 격자 지점을 사용
nearest_node = ox.distance.nearest_nodes(G, example_point[0], example_point[1])
nearest_edge = ox.distance.nearest_edges(G, example_point[0], example_point[1])

print(f"Nearest node to point {example_point}: {nearest_node}")
print(f"Nearest edge to point {example_point}: {nearest_edge}")

fig, ax = ox.plot_graph(G, show=False, close=False)
ax.scatter(*example_point, c='red', s=100, zorder=5)  # 격자 지점
ax.scatter(G.nodes[nearest_node]['x'], G.nodes[nearest_node]['y'], c='blue', s=100, zorder=5)  # 가장 가까운 노드
plt.show()

In [ ]:
import networkx as nx
import osmnx as ox
import numpy as np

# 그래프 생성 및 투영
G = ox.graph_from_place('서교동, 마포구, 서울, 대한민국', network_type='walk')
G_proj = ox.project_graph(G)

# 교차로 통합
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=False)
G2 = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

# 간선의 수 구하기
edges_count_G = G.number_of_edges()
edges_count_G2 = G2.number_of_edges()

# 결과 출력
print("Intersections: {}, G: {}, G2: {}, Edges in G: {}, Edges in G2: {}".format(len(intersections), len(G), len(G2), edges_count_G, edges_count_G2))

# 노드에 라벨링 추가
node_labels = {node: str(i) for i, node in enumerate(G2.nodes())}

# 노드와 엣지를 GeoDataFrame으로 변환
nodes, edges = ox.graph_to_gdfs(G2)
bbox = nodes.total_bounds  # 경계를 구합니다: (minx, miny, maxx, maxy)

# 격자 크기 설정
cell_size = 0.01  # 격자 크기 (단위: 도)
x_min, y_min, x_max, y_max = bbox
x_points = np.arange(x_min, x_max, cell_size)
y_points = np.arange(y_min, y_max, cell_size)

# 격자 지점을 일부분씩 처리
chunk_size = 10  # 한 번에 처리할 격자 지점 수
grid_points = [(x, y) for x in x_points for y in y_points]

for i in range(0, len(grid_points), chunk_size):
    chunk = grid_points[i:i + chunk_size]
    for example_point in chunk:
        nearest_node = ox.distance.nearest_nodes(G2, example_point[0], example_point[1])
        nearest_edge = ox.distance.nearest_edges(G2, example_point[0], example_point[1])

        # 라벨링된 노드 번호 출력
        nearest_node_label = node_labels[nearest_node]

        print(f"Example point: {example_point}")
        print(f"Nearest node to point {example_point}: {nearest_node} (Label: {nearest_node_label})")
        print(f"Nearest edge to point {example_point}: {nearest_edge}")


In [ ]:
# 격자 가시화
fig, ax = ox.plot_graph(G, show=False, close=False)

for x in x_points:
    ax.plot([x, x], [y_min, y_max], color='gray', linestyle='--', linewidth=0.5)

for y in y_points:
    ax.plot([x_min, x_max], [y, y], color='gray', linestyle='--', linewidth=0.5)

plt.show()

point 지정해서 가장 가까운 노드 엣지 찾기

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point

# 그래프 생성 및 투영
G = ox.graph_from_place('서교동, 마포구, 서울, 대한민국', network_type='walk')
G_proj = ox.project_graph(G)

# 교차로 통합
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=False)
G2 = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

# 간선의 수 구하기
edges_count_G = G.number_of_edges()
edges_count_G2 = G2.number_of_edges()

# 결과 출력
print(f"Intersections: {len(intersections)}, G: {len(G)}, G2: {len(G2)}, Edges in G: {edges_count_G}, Edges in G2: {edges_count_G2}")

# 노드에 라벨링 추가
node_labels = {node: str(i) for i, node in enumerate(G2.nodes())}

# 그래프 시각화
fig, ax = ox.plot_graph(G2, show=False, close=False)

# 노드 위치 가져오기
pos = {node: (data['x'], data['y']) for node, data in G2.nodes(data=True)}

# 노드 라벨링 추가
for node, (x, y) in pos.items():
    ax.text(x, y, node_labels[node], fontsize=7, ha='center', va='center', color='yellow')

plt.show()

# 예시 포인트 선택 및 좌표 변환
example_point = Point(126.9248, 37.5512)  # 서교동 중심부의 좌표 예시 (경도, 위도)
example_point_proj = ox.projection.project_geometry(example_point, to_crs=G2.graph['crs'])[0]

# 네트워크에서 가장 가까운 노드와 엣지 찾기
nearest_node = ox.distance.nearest_nodes(G2, X=example_point_proj.x, Y=example_point_proj.y)
nearest_edge = ox.distance.nearest_edges(G2, X=example_point_proj.x, Y=example_point_proj.y)

# 라벨링된 노드 번호 출력
nearest_node_label = node_labels[nearest_node]

print(f"Example point: {example_point} -> Projected point: {example_point_proj}")
print(f"Nearest node to point {example_point}: {nearest_node} (Label: {nearest_node_label})")
print(f"Nearest edge to point {example_point}: {nearest_edge}")


고도 구하기 (지형 반영)

In [ ]:
url = "https://maps.googleapis.com/maps/api/elevation/json?locations=39.7391536%2C-104.9847034&key=YOUR_API_KEY"
response = requests.get(url)
results = response.json().get('results')
if results:
print(results[0]['elevation'])

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point
import requests
import pandas as pd
import time
from pyproj import Transformer

# Google Elevation API 키 설정
api_key = 'YOUR_API_KEY'

def get_elevation(lat, lon, api_key):
  url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat}%2C-{lon}&key={api_key}"
  try:
      response = requests.get(url)
      response.raise_for_status()  # 요청이 성공적으로 완료되지 않으면 예외 발생
      results = response.json().get('results')
      if results:
          return results[0]['elevation']
      else:
          print(f"No elevation data for point: {lat}, {lon}")
          return None
  except requests.RequestException as e:
      print(f"Request failed for point {lat}, {lon}: {e}")
      return None

def add_elevation_data(G, api_key):
  for node, data in G.nodes(data=True):
      lon, lat = transformer.transform(data['x'], data['y'])
      elevation = get_elevation(lat, lon, api_key)
      if elevation is not None:
          G.nodes[node]['elevation'] = elevation
      else:
          G.nodes[node]['elevation'] = 0  # 기본값으로 0 설정
      time.sleep(0.1)  # API 요청 간 짧은 지연시간 추가

def calculate_edge_grades(G):
  for u, v, data in G.edges(data=True):
      elevation_u = G.nodes[u].get('elevation', 0)
      elevation_v = G.nodes[v].get('elevation', 0)
      length = data['length']
      if length > 0:
          grade = (elevation_v - elevation_u) / length
          G.edges[u, v, 0]['grade'] = grade

# 그래프 생성 및 투영
G = ox.graph_from_place('서교동, 마포구, 서울, 대한민국', network_type='walk')
G_proj = ox.project_graph(G)

# Google Elevation API를 사용하여 고도 데이터 추가
add_elevation_data(G_proj, api_key)

# 각 엣지의 경사도 계산
calculate_edge_grades(G_proj)

# 교차로 통합
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=False)
G2 = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

# 경사도 색상 설정
edge_colors = ox.plot.get_edge_colors_by_attr(G2, attr='grade', cmap='viridis')

# 그래프 시각화
fig, ax = ox.plot_graph(G2, edge_color=edge_colors, edge_linewidth=2, node_size=0, bgcolor='k')

# 예시 포인트 선택 및 좌표 변환
example_point = Point(126.9248, 37.5512)  # 서교동 중심부의 좌표 예시 (경도, 위도)

In [ ]:
from pyproj import Transformer
nodes, edges = ox.graph_to_gdfs(G2)

# UTM 좌표계를 경위도로 변환할 수 있는 Transformer를 생성합니다.
utm_zone = nodes.crs.to_string()  # 노드의 현재 좌표계
transformer = Transformer.from_crs(utm_zone, "epsg:4326", always_xy=True)

# GeoDataFrame을 사용하여 노드의 좌표를 확인합니다 (UTM 좌표계)
print(nodes[['y', 'x']].head())

# 노드의 좌표를 경위도로 변환하여 출력합니다
for node, data in G2.nodes(data=True):
  lat, lon = transformer.transform(data['x'], data['y'])
  print(f"Node {node}: {lat}, {lon}")
   37.5513896, 126.9165789

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point
import requests
import pandas as pd
import time
from pyproj import Transformer

# Google Elevation API 키 설정
api_key = 'YOUR_API_KEY'

def get_elevation(lat, lon, api_key):
  url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat}%2C{lon}&key={api_key}"
  try:
      response = requests.get(url)
      response.raise_for_status()  # 요청이 성공적으로 완료되지 않으면 예외 발생
      results = response.json().get('results')
      if results:
          print(results[0]['elevation'])
          return results[0]['elevation']
      else:
          print(f"No elevation data for point: {lat}, {lon}")
          return None
  except requests.RequestException as e:
      print(f"Request failed for point {lat}, {lon}: {e}")
      return None

def add_elevation_data(G, api_key):
  for node, data in G.nodes(data=True):
      lon, lat = transformer.transform(data['x'], data['y'])
      elevation = get_elevation(lat, lon, api_key)
      if elevation is not None:
          G.nodes[node]['elevation'] = elevation
      else:
          G.nodes[node]['elevation'] = 0  # 기본값으로 0 설정
      time.sleep(0.1)  # API 요청 간 짧은 지연시간 추가

def calculate_edge_grades(G):
  for u, v, data in G.edges(data=True):
      elevation_u = G.nodes[u].get('elevation', 0)
      elevation_v = G.nodes[v].get('elevation', 0)
      length = data['length']
      if length > 0:
          grade = (elevation_v - elevation_u) / length
          G.edges[u, v, 0]['grade'] = grade

# 그래프 생성 및 투영
G = ox.graph_from_place('서교동, 마포구, 서울, 대한민국', network_type='walk')
G_proj = ox.project_graph(G)

# Google Elevation API를 사용하여 고도 데이터 추가
add_elevation_data(G_proj, api_key)

# 각 엣지의 경사도 계산
calculate_edge_grades(G_proj)

# 교차로 통합
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=False)
G2 = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

# 경사도 색상 설정
edge_colors = ox.plot.get_edge_colors_by_attr(G2, attr='grade', cmap='viridis')
edge_colors = edge_colors.reset_index(drop=True)
# 그래프 시각화
fig, ax = ox.plot_graph(G2, edge_color=edge_colors, edge_linewidth=2, node_size=0, bgcolor='k')

# 예시 포인트 선택 및 좌표 변환
example_point = Point(126.9248, 37.5512)  # 서교동 중심부의 좌표 예시 (경도, 위도)


카페( 200m ~ 300m 사이의 노드들 구하기)

In [ ]:
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
import requests
import folium

# 지도 스타일 설정
ox.settings.log_console = True
ox.settings.use_cache = True

# 특정 장소의 도보 네트워크 그래프를 생성합니다.
place_name = '서교동,마포구, 서울, 대한민국'
G = ox.graph_from_place(place_name, network_type='walk')
G_proj = ox.project_graph(G)
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=True)
G2 = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

# 간선의 수 구하기
edges_count_G = G.number_of_edges()
edges_count_G2 = G2.number_of_edges()

# 결과 출력
print(f"Intersections: {len(intersections)}, G: {len(G)}, G2: {len(G2)}, Edges in G: {edges_count_G}, Edges in G2: {edges_count_G2}")

# 노드에 라벨링 추가
node_labels = {node: str(i) for i, node in enumerate(G2.nodes())}

##############################################################################################
# CSV 파일 경로
file_path = '/content/소상공인시장진흥공단_상가(상권)정보_서울_202403.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path, encoding='cp949')

# 필요한 열만 선택
df = df[['상호명', '도로명주소']]

# 구글 지도 API 키 설정
GOOGLE_MAPS_API_KEY = 'YOUR_API_KEY'

# 구글 지도 API를 이용한 주소 검색 함수
def search_address(row):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={row['도로명주소']}&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            return {
                '상호명': row['상호명'],
                '주소': row['도로명주소'],
                '위도': result['geometry']['location']['lat'],
                '경도': result['geometry']['location']['lng'],
                '전체 주소': result['formatted_address']
            }
    return None

# 검색 결과를 저장할 리스트
results = []

# 각 주소를 검색하여 결과를 리스트에 추가
for _, row in df.iterrows():
    result = search_address(row)
    if result:
        results.append(result)

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 모든 노드의 위치 가져오기
nodes = G2.nodes(data=True)

# 거리 계산 함수
def get_nodes_within_distance(center_point_proj, min_dist, max_dist, nodes):
  filtered_nodes = []
  for node, data in nodes:
      node_proj = Point(data['x'], data['y'])
      distance = center_point_proj.distance(node_proj)
      if min_dist <= distance <= max_dist:
          filtered_nodes.append(node)
  return filtered_nodes

# 모든 간선의 가중치를 0으로 초기화
for u, v, key, data in G2.edges(keys=True, data=True):
    data['weight'] = 0

################################################################################################
# 중심점 좌표 (위도, 경도)
for center_point in results_df[['위도', '경도']].values:

  # 중심점을 그래프의 투영 좌표계로 변환
  center_point_proj = ox.projection.project_geometry(Point(center_point[1], center_point[0]), to_crs=G_proj.graph['crs'])[0]
################################################################################################

  # 200m ~ 300m 사이의 노드 필터링
  filtered_nodes = get_nodes_within_distance(center_point_proj, 200, 300, nodes)

  # 필터링된 노드 라벨링 출력
  filtered_node_labels = [node_labels[node] for node in filtered_nodes]
  #print(f"Filtered nodes (200m ~ 300m): {filtered_node_labels}")

  # 필터링된 노드들이 포함된 엣지 찾기
  filtered_edges = [(u, v) for u, v in G2.edges() if u in filtered_nodes or v in filtered_nodes]

  # 필터링된 엣지들의 가중치를 1씩 증가
  for u, v in filtered_edges:
      weight_increase = np.random.normal(250, 100)
      G2[u][v][0]['weight'] += weight_increase

# 모든 간선들의 가중치 출력
for u, v, data in G2.edges(data=True):
    if(u<v):
      print(f"Edge ({u}, {v}): Weight = {data['weight']:.2f}")

가중치 기준 정하기

- 인구 밀집 지역 :가중치 1
- 공공시설물 좌표(소형) → 가장 가까운  간선 번호 구하기 (clear):4.0 (True/False) 가중치 1
- 카페 좌표(주소) → 좌표 기준 반경 200~300 m 사이에 있는 간선에 가중치++ (1.0): 가중치 0.9
- 공공시설물 좌표(대형) → 얼마 없어서 실제로 노가다:(True/False) 가중치 0.6
- 쓰레기통 간의 거리(한 간선에 두개 이상 있으면 마이너스): 두 개 이상일 때 개수당 -1
- 경사 → 각 노드의 고도 구함: 가중치 -0.3

우리 나름대로의 기준을 세우자!

최대 2 정도 > 0.5 곱해서 1로 만든다.

4 100

200~300안에 dist가 형성된다면 >> abs(dist-250)

- 인구 밀집
score_population_density = sigmoid((population_density - 1000) / 500)

$\sum{a_i*w_i}$ (a가 각 간선들 w가 각 가중치 )

score_cafe_distance_normalized =sigmoid((cafe_distance-250)/100)

가

score_population_density = gaussian(population_density, mean_population_density, std_population_density)
from pyproj import Transformer
nodes, edges = ox.graph_to_gdfs(G2)

# UTM 좌표계를 경위도로 변환할 수 있는 Transformer를 생성합니다.
utm_zone = nodes.crs.to_string()  # 노드의 현재 좌표계
transformer = Transformer.from_crs(utm_zone, "epsg:4326", always_xy=True)

line = data['geometry']
midpoint = line.interpolate(0.5, normalized=True)
midpoint_latlon = transformer.transform(midpoint.x, midpoint.y)

In [ ]:
def gaussian(x, mean, std):
return np.exp(-0.5 * ((x - mean) / std) ** 2) / (std * np.sqrt(2 * np.pi))

def add_edge_weights_based_on_distance(G, reference_point):
for u, v, data in G.edges(data=True):
  u_point = (G.nodes[u]['x'], G.nodes[u]['y'])
  v_point = (G.nodes[v]['x'], G.nodes[v]['y'])
  
  # 좌표 변환 (osmnx 좌표계 -> 경위도 좌표계)
  u_point_latlon = transformer.transform(u_point[0], u_point[1])
  v_point_latlon = transformer.transform(v_point[0], v_point[1])

  # u, v 중간 지점의 좌표를 사용
  midpoint = ((u_point_latlon[1] + v_point_latlon[1]) / 2, (u_point_latlon[0] + v_point_latlon[0]) / 2)
  distance = geodesic(reference_point, midpoint).meters  
  
  # 간선의 가중치를 거리로 설정
  if(200<=distance<=300):
    data['weight'] += gaussian(distance,250,100)
	
return G

reference_point = (37.5665, 126.9780)  # 서울 시청의 좌표
G3 = add_edge_weights_based_on_distance(G2, reference_point)

In [ ]:
for u, v, data in G3.edges(data=True):
    print(f"Edge from {u} to {v} has weight {data['weight']:.2f} meters")

In [ ]:
#모든 기준을 정규분포 상 0~1 사이로 가지게 하고 우리가 정해준 차등 기준에 따라 한번 더 곱해서 더하는 과정
def calculate_edge_grades(G):
  for u, v, data in G.edges(data=True):
      elevation_u = G.nodes[u].get('elevation', 0)
      elevation_v = G.nodes[v].get('elevation', 0)
      length = data['length']
      if length > 0:
          grade = (elevation_v - elevation_u) / length
          G.edges[u, v, 0]['grade'] = grade

In [ ]:
import networkx as nx
import osmnx as ox
from shapely.geometry import Point
from scipy.stats import norm

# 거리 계산 함수
def get_nodes_within_distance(center_point_proj, min_dist, max_dist, nodes):
    filtered_nodes = []
    for node, data in nodes:
        node_proj = Point(data['x'], data['y'])
        distance = center_point_proj.distance(node_proj)
        if min_dist <= distance <= max_dist:
            filtered_nodes.append((node, distance))
    return filtered_nodes

# 모든 간선의 가중치를 0으로 초기화
for u, v, key, data in G2.edges(keys=True, data=True):
    data['weight'] = 0

# 중심점 좌표 (위도, 경도)
for center_point in results_df[['위도', '경도']].values:

    # 중심점을 그래프의 투영 좌표계로 변환
    center_point_proj = ox.projection.project_geometry(Point(center_point[1], center_point[0]), to_crs=G_proj.graph['crs'])[0]

    # 200m ~ 300m 사이의 노드 필터링
    filtered_nodes = get_nodes_within_distance(center_point_proj, 200, 300, nodes)

    # 필터링된 노드 라벨링 출력
    filtered_node_labels = [node_labels[node] for node, _ in filtered_nodes]
    #print(f"Filtered nodes (200m ~ 300m): {filtered_node_labels}")

    # 필터링된 노드들이 포함된 엣지 찾기 및 가중치 계산
    for node, distance in filtered_nodes:
        weight = norm.pdf(distance, loc=250, scale=100)
        for neighbor in G2.neighbors(node):
            G2[node][neighbor][0]['weight'] += weight

# 모든 간선들의 가중치 출력
for u, v, data in G2.edges(data=True):
    if u < v:
        print(f"Edge ({u}, {v}): Weight = {data['weight']:.2f}")

쓰레기통이 배치된 간선 추출

In [ ]:
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import pandas as pd
import requests
import folium
import time

# 지도 스타일 설정
ox.settings.log_console = True
ox.settings.use_cache = True

# 특정 장소의 도보 네트워크 그래프를 생성합니다.
place_name = '서교동,마포구, 서울, 대한민국'
G = ox.graph_from_place(place_name, network_type='walk')
G_proj = ox.project_graph(G)
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=True)
G2 = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

# 간선의 수 구하기
edges_count_G = G.number_of_edges()
edges_count_G2 = G2.number_of_edges()

# 결과 출력
print(f"Intersections: {len(intersections)}, G: {len(G)}, G2: {len(G2)}, Edges in G: {edges_count_G}, Edges in G2: {edges_count_G2}")

# 노드에 라벨링 추가
node_labels = {node: str(i) for i, node in enumerate(G2.nodes())}

# 쓰레기통 위치 데이터 로드
trash_bins_file_path = 'C:\\Users\\sonym\\Documents\\서울특별시 마포구 가로쓰레기통현황 (1).csv'
trash_bins_df = pd.read_csv(trash_bins_file_path, encoding='cp949')

# 구글 지도 API 키 설정
GOOGLE_MAPS_API_KEY = 'YOUR_API_KEY'

# 구글 지도 API를 이용한 주소 검색 함수
def geocode_address(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            return (result['geometry']['location']['lat'], result['geometry']['location']['lng'])
    return None

# 쓰레기통 주소를 검색하여 결과를 리스트에 추가
def geocode_addresses(df):
    results = []
    for _, row in df.iterrows():
        result = geocode_address(row['도로명주소'])
        if result:
            results.append(result)
            time.sleep(0.1)  # 요청 간의 간격을 두어 API 사용 제한에 걸리지 않도록 함
    return results

# 쓰레기통 위치 데이터 지오코딩
trash_bins_coords = geocode_addresses(trash_bins_df)

# 지오코딩 결과 확인
print(trash_bins_coords)

# 거리 계산 함수
def get_nodes_within_distance(center_point_proj, max_dist, nodes):
    filtered_nodes = []
    for node, data in nodes:
        node_proj = Point(data['x'], data['y'])
        distance = center_point_proj.distance(node_proj)
        if distance <= max_dist:
            filtered_nodes.append(node)
    return filtered_nodes

# 쓰레기통이 포함된 간선들을 찾기
edges_with_trash_bins = set()
for center_point in trash_bins_coords:
    # 중심점을 그래프의 투영 좌표계로 변환
    center_point_proj = ox.projection.project_geometry(Point(center_point[1], center_point[0]), to_crs=G_proj.graph['crs'])[0]

    # 50m 이내의 노드 필터링
    filtered_nodes = get_nodes_within_distance(center_point_proj, 50, G2.nodes(data=True))

    # 필터링된 노드들이 포함된 엣지 찾기
    for node in filtered_nodes:
        for neighbor in G2.neighbors(node):
            if G2.has_edge(node, neighbor):
                edges_with_trash_bins.add((node, neighbor))

# 쓰레기통이 포함된 간선들을 지도에 시각화
m = folium.Map(location=[trash_bins_coords[0][0], trash_bins_coords[0][1]], zoom_start=15)

# 쓰레기통이 포함된 간선들을 지도에 추가 (색상과 두께 조정)
for u, v in edges_with_trash_bins:
    folium.PolyLine(
        locations=[(G2.nodes[u]['y'], G2.nodes[u]['x']), (G2.nodes[v]['y'], G2.nodes[v]['x'])],
        color='red', 
        weight=10,  # 더 굵게 설정
        opacity=1  # 더 진하게 설정
    ).add_to(m)

# 지도를 HTML 파일로 저장
m.save('C:\\Users\\sonym\\Documents\\trash_bins_edges_map.html')

m


쓰레기통 배치에 따라 겹치는 쓰레기통 뽑아내기

In [ ]:
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt



GOOGLE_MAPS_API_KEY = 'YOUR_API_KEY'


# CSV 파일 경로 설정
trash_bins_file_path = r"C:\Users\sonym\Documents\서울특별시 마포구 가로쓰레기통현황 (1).csv"

# CSV 파일을 'cp949' 인코딩으로 불러오기
trash_bins_df = pd.read_csv(trash_bins_file_path, encoding='cp949')

# 특정 장소의 도보 네트워크 그래프를 생성
place_name = '서교동,마포구, 서울, 대한민국'
G = ox.graph_from_place(place_name, network_type='walk')
G_proj = ox.project_graph(G)

# 교차로 통합
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=False)
G2 = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

# 쓰레기통 주소를 검색하여 결과를 리스트에 추가
def geocode_address(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            return (result['geometry']['location']['lat'], result['geometry']['location']['lng'])
    return None

def geocode_addresses(df):
    results = []
    for _, row in df.iterrows():
        result = geocode_address(row['도로명주소'])
        if result:
            results.append(result)
            time.sleep(0.1)
    return results

# 쓰레기통 위치 데이터 지오코딩
trash_bins_coords = geocode_addresses(trash_bins_df)

# 노드에 라벨링 추가
node_labels = {node: str(i) for i, node in enumerate(G2.nodes())}

# 그래프 시각화
fig, ax = ox.plot_graph(G2, show=False, close=False)

# 노드 위치 가져오기
pos = {node: (data['x'], data['y']) for node, data in G2.nodes(data=True)}

# 노드 라벨링 추가
for node, (x, y) in pos.items():
    ax.text(x, y, node_labels[node], fontsize=7, ha='center', va='center', color='yellow')

plt.show()

# 각 간선에 대해 겹치는 쓰레기통 수를 저장할 딕셔너리
edge_trash_bin_counts = {}

# 각 쓰레기통 위치에 대해 가장 가까운 간선을 찾고, 해당 간선의 쓰레기통 수를 증가
for lat, lon in trash_bins_coords:
    point = Point(lon, lat)
    point_proj = ox.projection.project_geometry(point, to_crs=G_proj.graph['crs'])[0]
    nearest_edge = ox.distance.nearest_edges(G2, X=point_proj.x, Y=point_proj.y)

    u, v, key = nearest_edge
    u_label, v_label = node_labels[u], node_labels[v]

    if (u_label, v_label, key) in edge_trash_bin_counts:
        edge_trash_bin_counts[(u_label, v_label, key)] += 1
    else:
        edge_trash_bin_counts[(u_label, v_label, key)] = 1

# 전체 쓰레기통 수 계산
total_trash_bins = len(trash_bins_coords)

# 겹치는 쓰레기통 수 계산 (최대 2로 제한)
capped_total_trash_bins = sum(min(count, 2) for count in edge_trash_bin_counts.values())

# 겹치는 쓰레기통 수 계산 (2개 이상 겹치는 경우 1로 계산)
overlapping_trash_bins = sum(1 for count in edge_trash_bin_counts.values() if count > 1)


print(f"Total trash bins with cap of 2 per edge: {capped_total_trash_bins}")
print(f"Total overlapping trash bins (more than 1 per edge): {overlapping_trash_bins}")

# 결과 출력
for edge, count in edge_trash_bin_counts.items():
    u_label, v_label, key = edge
    print(f"Edge from {u_label} to {v_label} has {min(count, 2)} trash bins")


유동 인구에 따라 집계구별 가중치

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# 데이터 읽기
hitmap_data_path = '/content/hitmap.csv'
hitmap_data = pd.read_csv(hitmap_data_path)

# 'geometry' 열을 WKT 형식에서 shapely 객체로 변환
hitmap_data['geometry'] = hitmap_data['geometry'].apply(wkt.loads)

# GeoDataFrame으로 변환
gdf = gpd.GeoDataFrame(hitmap_data, geometry='geometry')

# 현재 좌표 체계 확인 (예: EPSG:5179)
print(f"Current CRS: {gdf.crs}")

# 좌표 체계 변환 (EPSG:4326)
gdf = gdf.set_crs('EPSG:5179')  # 현재 좌표 체계를 설정 (예: EPSG:5179)
gdf = gdf.to_crs('EPSG:4326')  # WGS84로 변환

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# 데이터 읽기
hitmap_data_path = '/content/hitmap.csv'
hitmap_data = pd.read_csv(hitmap_data_path)

# 'geometry' 열을 WKT 형식에서 shapely 객체로 변환
hitmap_data['geometry'] = hitmap_data['geometry'].apply(wkt.loads)

# GeoDataFrame으로 변환
gdf = gpd.GeoDataFrame(hitmap_data, geometry='geometry')

# 현재 좌표 체계 확인 (예: EPSG:5179)
print(f"Current CRS: {gdf.crs}")

# 좌표 체계 변환 (EPSG:4326)
gdf = gdf.set_crs('EPSG:5179')  # 현재 좌표 체계를 설정 (예: EPSG:5179)
gdf = gdf.to_crs('EPSG:4326')  # WGS84로 변환

In [ ]:
################  MinMax 표준화 코드  ###################

import networkx as nx
from sklearn.preprocessing import MinMaxScaler
import numpy as np

weights = np.array([d['weight'] for u, v, d in G2.edges(data=True)])
scaler = MinMaxScaler()
normalized_weights = scaler.fit_transform(weights.reshape(-1, 1)).flatten()

for i, (u, v, d) in enumerate(G2.edges(data=True)):
  d['weight'] = normalized_weights[i]

for u, v, d in G2.edges(data=True):
  print(f"Edge ({u}, {v}) - Normalized weight: {d['weight']}")

In [ ]:
from pyproj import Transformer
nodes, edges = ox.graph_to_gdfs(G2)

# UTM 좌표계를 경위도로 변환할 수 있는 Transformer를 생성합니다.
utm_zone = nodes.crs.to_string()  # 노드의 현재 좌표계
transformer = Transformer.from_crs(utm_zone, "epsg:4326", always_xy=True)

# 노드의 좌표를 경위도로 변환하여 출력합니다
for node, data in G2.nodes(data=True):
  lat, lon = transformer.transform(data['x'], data['y'])
  print(f"Node {node}: {lat}, {lon}")
  
def get_elevation(lat, lon, api_key):
  url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat}%2C{lon}&key={api_key}"
  try:
      response = requests.get(url)
      response.raise_for_status()  # 요청이 성공적으로 완료되지 않으면 예외 발생
      results = response.json().get('results')
      if results:
          return results[0]['elevation']
      else:
          print(f"No elevation data for point: {lat}, {lon}")
          return None
  except requests.RequestException as e:
      print(f"Request failed for point {lat}, {lon}: {e}")
      return None

def add_elevation_data(G, api_key):
  for node, data in G.nodes(data=True):
      lon, lat = transformer.transform(data['x'], data['y'])
      elevation = get_elevation(lat, lon, api_key)
      if elevation is not None:
          G.nodes[node]['elevation'] = elevation
      else:
          G.nodes[node]['elevation'] = 0  # 기본값으로 0 설정
      time.sleep(0.1)  # API 요청 간 짧은 지연시간 추가

def calculate_edge_grades(G):
  for u, v, data in G.edges(data=True):
      elevation_u = G.nodes[u].get('elevation', 0)
      elevation_v = G.nodes[v].get('elevation', 0)
      length = data['length']
      if length > 0:
          grade = (elevation_v - elevation_u) / length
          G.edges[u, v, 0]['grade'] = grade
          # 간선의 가중치에 고도 정보 더하기
          data['weight'] = abs(grade)

# 노드에 고도 정보 추가
add_elevation_data(G2, api_key)

# 모든 간선의 가중치를 0으로 초기화
for u, v, key, data in G2.edges(keys=True, data=True):
    data['weight'] = 0

# 간선의 경사 계산 및 가중치에 추가
calculate_edge_grades(G2)

# 간선의 가중치 출력
for u, v, data in G2.edges(data=True):
    print(f"Edge ({u}, {v}): Weight = {data['weight']:.4f}")

In [ ]:
######## 가중치 합 구하기 ########
G_total = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)    #교차점 합치기(간략화된 지도)

# 모든 간선의 가중치를 0으로 초기화
for u, v, key, data in G_total.edges(keys=True, data=True):
    data['weight'] = 0

# G_bus, G_cafe, G_elev, G_pop의 가중치를 다 더한 걸 가중치로 사용
def add_weights_to_total(G_source, weight_factor):
    for u, v, key, data in G_source.edges(keys=True, data=True):
        if G_total.has_edge(u, v, key):
            G_total.edges[u, v, key]['weight'] += data['weight'] * weight_factor
        else:
            G_total.add_edge(u, v, key=key, weight=data['weight'] * weight_factor)

add_weights_to_total(G_bus, 1)
add_weights_to_total(G_cafe, 0.9)
add_weights_to_total(G_elev, -0.3)
add_weights_to_total(G_pop, 1)

# G_total 가중치 출력
for u, v, d in G_total.edges(data=True):
  print(f"Edge ({u}, {v}) - weight: {d['weight']}")

In [ ]:
######################## 상위 103개 간선 가중치 합 #######################
import heapq

# 모든 간선의 가중치 추출
edges_with_weights = []
for u, v, key, data in G_total.edges(keys=True, data=True):
    edges_with_weights.append((data['weight'], u, v, key))

# 모든 간선의 가중치 추출 및 중복 간선 제거
edge_dict = {}
for u, v, key, data in G_total.edges(keys=True, data=True):
    # (u, v)와 (v, u)를 동일한 간선으로 간주하기 위해 튜플 정렬
    edge = tuple(sorted((u, v)))
    if edge not in edge_dict or edge_dict[edge][0] < data['weight']:
        edge_dict[edge] = (data['weight'], u, v, key)

# 간선 가중치를 기준으로 정렬하여 상위 103개의 간선 선택
top_edges = heapq.nlargest(103, edge_dict.values(), key=lambda x: x[0])

# 최적의 위치 (간선 정보 출력) 및 가중치 합 계산
total_weight = 0
for weight, u, v, key in top_edges:
    total_weight += weight
    print(f"Edge ({u}, {v}, {key}) - weight: {weight}")

print(f"Total weight of top 103 edges: {total_weight}")

최총 4단계

In [ ]:
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import pandas as pd
import requests
import time
from pyproj import Transformer
import matplotlib.pyplot as plt


# CSV 파일 경로 설정
trash_bins_file_path = 'C:\\Users\\sonym\\Documents\\서울특별시 마포구 가로쓰레기통현황 (1).csv'

# CSV 파일을 'cp949' 인코딩으로 불러오기
trash_bins_df = pd.read_csv(trash_bins_file_path, encoding='cp949')

# 필요한 열만 선택 (쓰레기통 데이터)
trash_bins_df = trash_bins_df[['도로명주소']]

# 구글 지도 API를 이용한 주소 검색 함수
def geocode_address(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            return (result['geometry']['location']['lat'], result['geometry']['location']['lng'])
    return None

# 쓰레기통 주소를 검색하여 결과를 리스트에 추가
def geocode_addresses(df):
    results = []
    for _, row in df.iterrows():
        result = geocode_address(row['도로명주소'])
        if result:
            results.append(result)
            time.sleep(0.1)  # 요청 간의 간격을 두어 API 사용 제한에 걸리지 않도록 함
    return results

# 쓰레기통 위치 데이터 지오코딩
trash_bins_coords = geocode_addresses(trash_bins_df)

# 지오코딩된 결과 확인
print("Geocoded trash bin coordinates:")
print(trash_bins_coords)

# OSMnx 설정
ox.settings.log_console = True
ox.settings.use_cache = True

# 특정 장소의 도보 네트워크 그래프를 생성
place_name = '서교동,마포구, 서울, 대한민국'
G = ox.graph_from_place(place_name, network_type='walk')
G_proj = ox.project_graph(G)
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=True)
G_total = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

# 모든 간선의 가중치를 0으로 초기화 (기존 가중치가 없는 경우)
for u, v, key, data in G_total.edges(keys=True, data=True):
    if 'weight' not in data:
        data['weight'] = 0

# 가중치를 더하는 함수 정의
def add_weights_to_total(G_source, weight_factor):
    for u, v, key, data in G_source.edges(keys=True, data=True):
        if G_total.has_edge(u, v, key):
            G_total.edges[u, v, key]['weight'] += data['weight'] * weight_factor
        else:
            G_total.add_edge(u, v, key=key, weight=data['weight'] * weight_factor)

# 가중치 추가 (예: G_bus, G_cafe, G_elev, G_pop, G_overlap 그래프들)
add_weights_to_total(G_bus, 1)
add_weights_to_total(G_cafe, 0.9)
add_weights_to_total(G_elev, -0.3)
add_weights_to_total(G_pop, 1)
add_weights_to_total(G_overlap, -0.5)

# 가중치가 있는 간선들을 출력
print("All edges in G_total with their weights:")
for u, v, data in G_total.edges(data=True):
    if data['weight'] > 0:
        print(f"Edge from {u} to {v} has weight {data['weight']:.4f}")

# UTM 좌표계를 경위도로 변환할 수 있는 Transformer를 생성합니다.
utm_crs = G_proj.graph['crs']
transformer = Transformer.from_crs("epsg:4326", utm_crs, always_xy=True)

# 각 쓰레기통 위치에 대해 가장 가까운 간선을 찾고, 해당 간선의 쓰레기통 수를 증가
edge_trash_bin_counts = {}

for lat, lon in trash_bins_coords:
    point_proj = Point(*transformer.transform(lon, lat))  # 경도, 위도 순서로 변환
    nearest_edge = ox.distance.nearest_edges(G_total, X=point_proj.x, Y=point_proj.y)

    u, v, key = nearest_edge
    if (u, v, key) in edge_trash_bin_counts:
        edge_trash_bin_counts[(u, v, key)] += 1
    else:
        edge_trash_bin_counts[(u, v, key)] = 1

# 전체 쓰레기통 수 계산
total_trash_bins = len(trash_bins_coords)

# 겹치는 쓰레기통 수 계산 (최대 2로 제한)
capped_total_trash_bins = sum(min(count, 2) for count in edge_trash_bin_counts.values())

print(f"Total number of trash bins: {total_trash_bins}")
print(f"Total number of capped trash bins: {capped_total_trash_bins}")

# 쓰레기통이 포함된 간선들의 가중치 합 계산 (최대 2개의 쓰레기통으로 제한)
total_weight_with_trash_bins = 0

for (u, v, key), count in edge_trash_bin_counts.items():
    total_weight_with_trash_bins += G_total[u][v][key]['weight'] * min(count, 2)

print(f"Total Weight of Edges with Trash Bins: {total_weight_with_trash_bins:.2f}")



In [ ]:
import heapq

# 모든 간선의 가중치를 0으로 초기화
for u, v, key, data in G_total.edges(keys=True, data=True):
    data['weight'] = 0

# G_bus, G_cafe, G_elev, G_pop의 가중치를 다 더한 걸 가중치로 사용
def add_weights_to_total(G_source, weight_factor):
    for u, v, key, data in G_source.edges(keys=True, data=True):
        if G_total.has_edge(u, v, key):
            G_total.edges[u, v, key]['weight'] += data['weight'] * weight_factor
        else:
            G_total.add_edge(u, v, key=key, weight=data['weight'] * weight_factor)

add_weights_to_total(G_bus, 1)
add_weights_to_total(G_cafe, 0.9)
add_weights_to_total(G_elev, -0.3)
add_weights_to_total(G_pop, 1)
# G_total 가중치 출력
for u, v, d in G_total.edges(data=True):
  print(f"Edge ({u}, {v}) - weight: {d['weight']}")

# 모든 간선의 가중치 추출
edges_with_weights = []
for u, v, key, data in G_total.edges(keys=True, data=True):
    edges_with_weights.append((data['weight'], u, v, key))

# 모든 간선의 가중치 추출 및 중복 간선 제거
edge_dict = {}
for u, v, key, data in G_total.edges(keys=True, data=True):
    # (u, v)와 (v, u)를 동일한 간선으로 간주하기 위해 튜플 정렬
    edge = tuple(sorted((u, v)))
    if edge not in edge_dict or edge_dict[edge][0] < data['weight']:
        edge_dict[edge] = (data['weight'], u, v, key)

# 가중치를 저장할 힙 생성
edge_heap = [[-weight, u, v, key] for weight, u, v, key in edge_dict.values()]
heapq.heapify(edge_heap)

total_weight = 0
filtered_edges = []

for _ in range(103):
    # 가중치가 가장 높은 간선 추출
    max_edge = heapq.heappop(edge_heap)
    weight = -max_edge[0]  # 힙에서 꺼낸 값을 다시 양수로 변환
    u, v, key = max_edge[1], max_edge[2], max_edge[3]
    
    # 최종 가중치 합에 추가
    total_weight += weight
    filtered_edges.append((u, v))
    
    # 선택된 간선의 가중치를 0.5 감소시키고 다시 힙에 삽입
    new_weight = weight - 0.5
    if new_weight > 0:
        heapq.heappush(edge_heap, [-new_weight, u, v, key])
        edge_dict[(u, v)] = [new_weight, u, v, key]

print(f"Total weight of top 103 edges: {total_weight}")

# 간선들을 시각화
fig, ax = plt.subplots(figsize=(12, 12))
ox.plot_graph(G_total, ax=ax, node_size=0, edge_color='gray', edge_linewidth=0.5, show=False, close=False)

for u, v in filtered_edges:
    x = [G_total.nodes[u]['x'], G_total.nodes[v]['x']]
    y = [G_total.nodes[u]['y'], G_total.nodes[v]['y']]
    ax.plot(x, y, color='red', linewidth=2, alpha=1.0)

plt.show()

In [ ]:
from scipy.spatial.distance import pdist

# 각 쓰레기통 위치에 대해 가장 가까운 간선을 찾고, 해당 간선의 쓰레기통 수를 증가
edge_trash_bin_counts = {}
edge_centroids = []

for lat, lon in trash_bins_coords:
    point_proj = Point(*transformer.transform(lon, lat))  # 경도, 위도 순서로 변환
    nearest_edge = ox.distance.nearest_edges(G_total, X=point_proj.x, Y=point_proj.y)

    u, v, key = nearest_edge
    if (u, v, key) in edge_trash_bin_counts:
        edge_trash_bin_counts[(u, v, key)] += 1
    else:
        edge_trash_bin_counts[(u, v, key)] = 1

    # 간선의 중심점 계산
    edge_centroid_x = (G_total.nodes[u]['x'] + G_total.nodes[v]['x']) / 2
    edge_centroid_y = (G_total.nodes[u]['y'] + G_total.nodes[v]['y']) / 2
    edge_centroids.append((edge_centroid_x, edge_centroid_y))



# 간선 중심점들 간의 거리 분산 계산
if edge_centroids:
    distances = pdist(edge_centroids)
    spread_variance = np.var(distances)
    print(f"Spread variance of edges with trash bins: {spread_variance:.4f}")
else:
    print("No edges with trash bins found.")

버스정류장 뉴 코드

In [ ]:
# CSV 파일 경로 설정
bus_stops_file_path = r"/content/서울특별시 마포구 버스정류소 위치정보_20230705.csv"

# CSV 파일을 'cp949' 인코딩으로 불러오기
bus_stops_df = pd.read_csv(bus_stops_file_path, encoding='cp949')

# 서교동 경계 좌표 범위 가져오기
bounds = ox.utils_geo.bbox_from_point((37.552992, 126.919967), dist=2000)  # 서교동 중심점과 거리
north, south, east, west = bounds[0], bounds[1], bounds[2], bounds[3]

# 서교동 경계 다각형 정의
seogyo_bounds = Polygon([
    (west, north),
    (east, north),
    (east, south),
    (west, south),
    (west, north)
])

# 서교동 지역의 버스 정류장만 선택
bus_stops_coords = bus_stops_df[['Y좌표', 'X좌표']].values
bus_stops_coords = [coord for coord in bus_stops_coords if seogyo_bounds.contains(Point(coord[1], coord[0]))]

# 구글 지도 API를 이용한 주소 검색 함수
def geocode_address(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            return (result['geometry']['location']['lat'], result['geometry']['location']['lng'])
    return None

# 모든 간선의 가중치를 초기화 (기존 가중치가 0인 경우)
for u, v, key, data in G_bus.edges(keys=True, data=True):
    data['weight'] = 0

def calculate_distance(point1, point2):
    return point1.distance(point2)

# 각 버스 정류장 위치에 대해 가장 가까운 간선을 찾고, 해당 간선의 가중치를 1씩 증가
for lat, lon in bus_stops_coords:
    point = Point(lon, lat)  # 경도, 위도 순서로 입력
    point_proj = ox.projection.project_geometry(point, to_crs=G2.graph['crs'])[0]
    nearest_edge = ox.distance.nearest_edges(G2, X=point_proj.x, Y=point_proj.y)

    u, v, key = nearest_edge
    
    # 노드의 좌표를 가져옴
    u_coord = Point(G2.nodes[u]['x'], G2.nodes[u]['y'])
    v_coord = Point(G2.nodes[v]['x'], G2.nodes[v]['y'])
    
    # 간선의 중간 지점을 계산
    mid_point = Point((u_coord.x + v_coord.x) / 2, (u_coord.y + v_coord.y) / 2)
    
    # 버스 정류장과 중간 지점 사이의 거리 계산
    distance_to_edge = calculate_distance(point_proj, mid_point)

    if distance_to_edge <= 60:  # 거리가 20m 이하인 경우에만 가중치를 증가
        if (u, v, key) in G_bus.edges(keys=True):
            G_bus[u][v][key]['weight'] = min(G_bus[u][v][key]['weight'] + 1, 2)
        else:
            print(f"KeyError: Edge ({u}, {v}, {key}) not found in G_bus")
    else:
        print(f"Distance to edge ({u}, {v}, {key}) is more than 20m: {distance_to_edge:.2f}m")

# 총 가중치 합 계산 (버스 정류장 가중치 합계)
total_weight = sum(data['weight'] for u, v, key, data in G_bus.edges(keys=True, data=True))
print(f"Total Weight of All Edges: {total_weight:.2f}")

# 가중치가 있는 간선들을 시각화
fig, ax = plt.subplots(figsize=(12, 12))
ox.plot_graph(G_bus, ax=ax, node_size=0, edge_color='gray', edge_linewidth=0.5, show=False, close=False)

# 가중치가 있는 간선들을 강조하여 추가 (색상과 두께 조정)
for u, v, data in G_bus.edges(data=True):
    if data['weight'] > 0 and (u, v) in G_bus.edges():
        x = [G_bus.nodes[u]['x'], G_bus.nodes[v]['x']]
        y = [G_bus.nodes[u]['y'], G_bus.nodes[v]['y']]
        # 두께를 조정하여 최대 두께를 제한
        ax.plot(x, y, color='red', linewidth=2, alpha=0.7)
        mid_x = (G_bus.nodes[u]['x'] + G_bus.nodes[v]['x']) / 2
        mid_y = (G_bus.nodes[u]['y'] + G_bus.nodes[v]['y']) / 2
        ax.text(mid_x, mid_y, str(data['weight']), fontsize=15, ha='center', va='center', color='black')

plt.show()

새로운 쓰레기통 

In [ ]:
# 각 간선에 대해 겹치는 쓰레기통 수를 저장할 딕셔너리
edge_trash_bin_counts = {}

def calculate_distance(point1, point2):
  return point1.distance(point2)

# 각 쓰레기통 위치에 대해 가장 가까운 간선을 찾고, 해당 간선의 쓰레기통 수를 증가
for lat, lon in trash_bins_coords:
  point = Point(lon, lat)
  point_proj = ox.projection.project_geometry(point, to_crs=G2.graph['crs'])[0]
  nearest_edge = ox.distance.nearest_edges(G_overlap, X=point_proj.x, Y=point_proj.y)

  u, v, key = nearest_edge
  
  # 노드의 좌표를 가져옴
  u_coord = Point(G2.nodes[u]['x'], G2.nodes[u]['y'])
  v_coord = Point(G2.nodes[v]['x'], G2.nodes[v]['y'])
  
  # 간선의 중간 지점을 계산
  mid_point = Point((u_coord.x + v_coord.x) / 2, (u_coord.y + v_coord.y) / 2)
  
  # 쓰레기통과 중간 지점 사이의 거리 계산
  distance_to_edge = calculate_distance(point_proj, mid_point)

  if distance_to_edge <= 50:  # 거리가 50m 이하인 경우에만 쓰레기통 수 증가
      u_label, v_label = node_labels[u], node_labels[v]

      if (u_label, v_label, key) in edge_trash_bin_counts:
          edge_trash_bin_counts[(u_label, v_label, key)] += 1
      else:
          edge_trash_bin_counts[(u_label, v_label, key)] = 1

# 전체 쓰레기통 수 계산
total_trash_bins = len(trash_bins_coords)

# 겹치는 쓰레기통 수 계산 
capped_total_trash_bins = sum(count for count in edge_trash_bin_counts.values())
# 겹치는 쓰레기통 수 계산 (2개 이상 겹치는 경우 1로 계산)
overlapping_trash_bins = sum(1 for count in edge_trash_bin_counts.values() if count > 1)

# 가중치 설정: 쓰레기통이 2개인 간선에는 1, 그렇지 않으면 0
for u, v, key, data in G_overlap.edges(keys=True, data=True):
  if (node_labels[u], node_labels[v], key) in edge_trash_bin_counts:
      data['weight'] = edge_trash_bin_counts[(node_labels[u], node_labels[v], key)]
  else:
      data['weight'] = 0

# 가중치가 있는 간선들을 출력
for u, v, data in G_overlap.edges(data=True):
  if data['weight'] >0:
      print(f"Edge from {u} to {v} has weight {data['weight']}")

In [ ]:
# 총 가중치 합 계산 
total_weight = sum(data['weight'] for u, v, key, data in G_overlap.edges(keys=True, data=True))
print(f"Total Weight of All Edges: {total_weight:.2f}")


# 가중치가 있는 간선들을 시각화
fig, ax = plt.subplots(figsize=(12, 12))
ox.plot_graph(G_bus, ax=ax, node_size=0, edge_color='gray', edge_linewidth=0.5, show=False, close=False)

# 가중치가 있는 간선들을 강조하여 추가 (색상과 두께 조정)
for u, v, data in G_overlap.edges(data=True):
  if data['weight'] > 0 and (u, v) in G_bus.edges():
      x = [G_bus.nodes[u]['x'], G_bus.nodes[v]['x']]
      y = [G_bus.nodes[u]['y'], G_bus.nodes[v]['y']]
      # 두께를 조정하여 최대 두께를 제한
      ax.plot(x, y, color='red', linewidth=2, alpha=0.7)
      mid_x = (G_bus.nodes[u]['x'] + G_bus.nodes[v]['x']) / 2
      mid_y = (G_bus.nodes[u]['y'] + G_bus.nodes[v]['y']) / 2
      ax.text(mid_x, mid_y, str(data['weight']), fontsize=15, ha='center', va='center', color='black')

plt.show()

In [ ]:
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import pandas as pd
import requests
import time
from pyproj import Transformer
import matplotlib.pyplot as plt


# CSV 파일 경로 설정
trash_bins_file_path = 'C:\\Users\\sonym\\Documents\\서울특별시 마포구 가로쓰레기통현황 (1).csv'

# CSV 파일을 'cp949' 인코딩으로 불러오기
trash_bins_df = pd.read_csv(trash_bins_file_path, encoding='cp949')

# 필요한 열만 선택 (쓰레기통 데이터)
trash_bins_df = trash_bins_df[['도로명주소']]

# 구글 지도 API를 이용한 주소 검색 함수
def geocode_address(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            result = data['results'][0]
            return (result['geometry']['location']['lat'], result['geometry']['location']['lng'])
    return None

# 쓰레기통 주소를 검색하여 결과를 리스트에 추가
def geocode_addresses(df):
    results = []
    for _, row in df.iterrows():
        result = geocode_address(row['도로명주소'])
        if result:
            results.append(result)
            time.sleep(0.1)  # 요청 간의 간격을 두어 API 사용 제한에 걸리지 않도록 함
    return results

# 쓰레기통 위치 데이터 지오코딩
trash_bins_coords = geocode_addresses(trash_bins_df)

# 지오코딩된 결과 확인
print("Geocoded trash bin coordinates:")
print(trash_bins_coords)

# OSMnx 설정
ox.settings.log_console = True
ox.settings.use_cache = True

# 특정 장소의 도보 네트워크 그래프를 생성
place_name = '서교동,마포구, 서울, 대한민국'
G = ox.graph_from_place(place_name, network_type='walk')
G_proj = ox.project_graph(G)
intersections = ox.consolidate_intersections(G_proj, rebuild_graph=False, tolerance=15, dead_ends=True)
G_total = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)

# 모든 간선의 가중치를 0으로 초기화 (기존 가중치가 없는 경우)
for u, v, key, data in G_total.edges(keys=True, data=True):
    if 'weight' not in data:
        data['weight'] = 0

# 가중치를 더하는 함수 정의
def add_weights_to_total(G_source, weight_factor):
    for u, v, key, data in G_source.edges(keys=True, data=True):
        if G_total.has_edge(u, v, key):
            G_total.edges[u, v, key]['weight'] += data['weight'] * weight_factor
        else:
            G_total.add_edge(u, v, key=key, weight=data['weight'] * weight_factor)

# 가중치 추가 (예: G_bus, G_cafe, G_elev, G_pop, G_overlap 그래프들)
add_weights_to_total(G_bus, 1)
add_weights_to_total(G_cafe, 0.9)
add_weights_to_total(G_elev, -0.3)
add_weights_to_total(G_pop, 1)
add_weights_to_total(G_overlap, -0.5)



# 가중치가 있는 간선들을 출력
print("All edges in G_total with their weights:")
for u, v, data in G_total.edges(data=True):
    if data['weight'] > 0:
        print(f"Edge from {u} to {v} has weight {data['weight']:.4f}")

# UTM 좌표계를 경위도로 변환할 수 있는 Transformer를 생성합니다.
utm_crs = G_proj.graph['crs']
transformer = Transformer.from_crs("epsg:4326", utm_crs, always_xy=True)

# 각 쓰레기통 위치에 대해 가장 가까운 간선을 찾고, 해당 간선의 쓰레기통 수를 증가
edge_trash_bin_counts = {}

for lat, lon in trash_bins_coords:
    point_proj = Point(*transformer.transform(lon, lat))  # 경도, 위도 순서로 변환
    nearest_edge = ox.distance.nearest_edges(G_total, X=point_proj.x, Y=point_proj.y)

    u, v, key = nearest_edge
    if (u, v, key) in edge_trash_bin_counts:
        edge_trash_bin_counts[(u, v, key)] += 1
    else:
        edge_trash_bin_counts[(u, v, key)] = 1

# 전체 쓰레기통 수 계산
total_trash_bins = len(trash_bins_coords)

# 겹치는 쓰레기통 수 계산 (최대 2로 제한)
capped_total_trash_bins = sum(min(count, 2) for count in edge_trash_bin_counts.values())

print(f"Total number of trash bins: {total_trash_bins}")
print(f"Total number of capped trash bins: {capped_total_trash_bins}")

# 쓰레기통이 포함된 간선들의 가중치 합 계산 (최대 2개의 쓰레기통으로 제한)
total_weight_with_trash_bins = 0

for (u, v, key), count in edge_trash_bin_counts.items():
    total_weight_with_trash_bins += G_total[u][v][key]['weight'] * min(count, 2)

print(f"Total Weight of Edges with Trash Bins: {total_weight_with_trash_bins:.2f}")

# 쓰레기통이 포함된 간선들을 시각화
fig, ax = plt.subplots(figsize=(12, 12))
ox.plot_graph(G_total, ax=ax, node_size=0, edge_color='gray', edge_linewidth=0.5, show=False, close=False)

for (u, v, key), count in edge_trash_bin_counts.items():
    if count > 0:
        x = [G_total.nodes[u]['x'], G_total.nodes[v]['x']]
        y = [G_total.nodes[u]['y'], G_total.nodes[v]['y']]
        ax.plot(x, y, color='red', linewidth=2 + min(count, 2), alpha=0.7)

plt.show()
